In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
from pathlib import Path

raw_data_folder: Path = Path("00_raw/")
prod_data_folder: Path = Path("30_prod/")

In [6]:
import pandas as pd
import numpy as np

daily_inflation_rate: pd.DataFrame = pd.read_csv(
    filepath_or_buffer=raw_data_folder.joinpath("T5YIE.csv"),
    sep=",",
    encoding="UTF-8",
)

daily_inflation_rate.rename(columns={"DATE": "Date", "T5YIE": "Rate"}, inplace=True)
daily_inflation_rate["Date"] = pd.to_datetime(daily_inflation_rate["Date"], format="%Y-%m-%d")

for i in range(daily_inflation_rate.shape[0]):
    if daily_inflation_rate.at[i, "Rate"] == ".":
        if i > 0 and i < daily_inflation_rate.shape[0] - 1:
            daily_inflation_rate.at[i, "Rate"] = round((float(daily_inflation_rate.at[i+1, "Rate"]) + float(daily_inflation_rate.at[i-1, "Rate"])) / 2, 2)
        elif i > 0:
            daily_inflation_rate.at[i, "Rate"] = float(daily_inflation_rate.at[i-1, "Rate"])
        elif i == 0:
            daily_inflation_rate.at[i, "Rate"] = float(daily_inflation_rate.at[i+1, "Rate"])

daily_inflation_rate["Rate"] = daily_inflation_rate["Rate"].astype(np.float64)

In [7]:
daily_inflation_rate.to_csv(
    path_or_buf=prod_data_folder.joinpath("inflation_rates.csv"),
    sep=",",
    encoding="UTF-8",
    index=False,
)